In [17]:
import pymysql
import json
import os


# Set this to "GCP" or "AWS" (or use an env var: DB_PLATFORM=GCP/AWS)
PLATFORM = os.getenv("DB_PLATFORM", "AWS").upper()

def getconn():
    return pymysql.connect(
    host="database-1.c9ikwq088lg0.us-east-2.rds.amazonaws.com",  # RDS endpoint = host
    port=3306, 
    user="root",                                   # MySQL default port      user="admin",
    password="Casey9203",
    database=None,
    ssl={"ca": "global-bundle.pem"},                         # or None if you CREATE first
)


def setup_db(cur):
  # Set up db
    cur.execute('CREATE DATABASE IF NOT EXISTS roster_db')
    cur.execute('USE roster_db')

    cur.execute('DROP TABLE IF EXISTS Member;')
    cur.execute('DROP TABLE IF EXISTS Course;')
    cur.execute('DROP TABLE IF EXISTS User;')



    cur.execute('''
        CREATE TABLE User (
        id     INT NOT NULL PRIMARY KEY AUTO_INCREMENT,
            name   VARCHAR(20) UNIQUE);
        ''');

    cur.execute('''CREATE TABLE Course (
            id     INT NOT NULL PRIMARY KEY AUTO_INCREMENT,
            title  VARCHAR(20) UNIQUE);
        ''');

    cur.execute('''CREATE TABLE Member (
            user_id     INT,
            course_id   INT,
            role        INT,
            FOREIGN KEY(user_id) REFERENCES User(id),
            FOREIGN KEY(course_id) REFERENCES Course(id),
            PRIMARY KEY (user_id, course_id));'''
        );


cnx = getconn()
cur = cnx.cursor()
print("Starting Setup...")
setup_db(cur)
print("Finished Setup.")
#print("Starting Insert...")
#insert_data(cur)
#print("Finished Insert.")
cur.close()
cnx.commit()
cnx.close()
print("FINISHED")


Starting Setup...
Finished Setup.
FINISHED
